# 들락날락 사이트에서 들락날락 시설명과 주소 가져오기
## 1. 들락날락 사이트에서 '우리동네 들락날락' -> '들락날락 정보'에 접속
- ### https://busan.go.kr/bschild/dnol/list.nm?menuCd=8&lang=ko&url=dnol
## 2. '들락날락 정보'에서 확인할 수 있는 106개의 들락날락 시설 중 하나를 클릭하여 각 들락날락 정보의 시설명과 주소가 해당하는 HTML 테그 찾기
- ### 예시로 반여도서관 들락날락을 클릭하고 f12를 눌러 개발자 도구를 열고 Ctrl+Shift+C를 눌러 원하는 데이터에 마우스를 대면 그 데이터가 해당하는 HTML 테그를 찾을 수 있습니다.
    - https://busan.go.kr/bschild/dnol/view.nm?menuCd=8&lang=ko&url=dnol&olSkey=DNOL0000000000086
    - '반여도서관 들락날락'의 HTML 테그는 <div class="con_title">반여도서관 들락날락</div>입니다.
    - 주소의 '해운대구 재반로282번길 38(반여동)'의 HTML 테그는 <td data-th="주소">해운대구 재반로282번길 38(반여동)</td>입니다.
    - 따라서 들락날락 시설명은 <div class="con_title"> 테그와 </div> 테그 사이의 문자열에 해당하고 들락날락의 주소는 <td data-th="주소"> 테그와 </td> 테그 사이의 문자열에 해당합니다.
## 3. 각각의 들락날락 정보의 URL 차이점 분석
- ### 사상구육아종합지원센터 들락날락
    - https://busan.go.kr/bschild/dnol/view.nm?menuCd=8&lang=ko&url=dnol&olSkey=DNOL0000000000064
- ### 국립해양박물관내 어린이해양도서관 들락날락(조성중)
    - https://busan.go.kr/bschild/dnol/view.nm?menuCd=8&lang=ko&url=dnol&olSkey=DNOL0000000000230
- ### URL의 차이점을 분석한 결과 차이가 있음을 확인
    - 'https://busan.go.kr/bschild/dnol/view.nm?menuCd=8&lang=ko&url=dnol&olSkey=DNOL0000000000'까지는 모두 같은 반면 맨 뒤에 숫자 3개가 다릅니다. 이 URL 뒤에 064를 붙이면 사상구육아종합지원센터 들락날락을, 230을 붙이면 국립해양박물관내 어린이해양도서관 들락날락(조성중)을 보여줍니다.
## 4. 이 차이점을 이용하여 들락날락 사이트에서 106개의 들락날락의 시설명과 주소를 한 번에 가져오는 코드 구상
- ### URL 맨 뒷자리 숫자 검색 범위는 0부터 300까지로 정했습니다.
    - ### 이유는 총 106개소의 들락날락이 있는데 각 들락날락 시설별로 URL을 0부터 105까지 혹은 1부터 106까지 설정했을 것이라는 예상과 달리 000, 001로 입력하면 404 에러가 뜨고 230 같이 106 이상의 숫자를 확인했기 때문입니다.
    - ### 그래서 106개소의 들락날락이 모두 포함될 것으로 예상되면서 과도한 접속으로 사이트에 부담을 주지 않는 범위로 0부터 300까지 설정하였습니다.

In [ ]:
%pip install requests
%pip install bs4
%pip install pandas
# 위 라이브러리들을 먼저 설치

In [ ]:
import requests
from bs4 import BeautifulSoup
# requests 라이브러리를 이용해서 웹 페이지를 가져오고 BeautifulSoup 라이브러리를 이용해서 HTML을 파싱하는 작업을 진행
import pandas as pd
# 수집한 데이터를 DataFrame 형태로 변환하기 위해 pandas 라이브러리 활용
import time
import random
# 과도한 요청으로 들락날락 서버에 부담을 주지 않기 위해
# 일정 시간 동안 쉬었다가 다시 URL에 접속할 수 있도록 time과 random 라이브러리 활용

data = []

for i in range(0, 301):
    response = requests.get(f'https://busan.go.kr/bschild/dnol/view.nm?menuCd=8&lang=ko&url=dnol&olSkey=DNOL0000000000{i:03d}')
    # requests.get() 메소드를 이용해서 웹 페이지를 가져오고
    # response 변수에 요청한 웹 페이지의 응답을 저장
    # response 변수에는 요청한 웹 페이지의 HTML 코드와 상태 코드가 포함되어 있다.
    # 이때 상태 코드는 요청이 성공했는지 실패했는지를 나타내는 코드이다.
    
    time.sleep(random.uniform(3, 5))
    # 3~5초 동안 쉬었다가 다시 요청하도록 설정
    
    if response.status_code != 200:
        print(f"olSkey=DNOL0000000000{i:03d}: 요청 실패 - {response.status_code}")
        continue
    # 만약 HTTP 응답 상태 코드가 200(정상)이 아니라면,
    # 해당 페이지의 데이터 수집을 건너뛰고(continue)
    # 실패한 olSkey와 상태 코드를 출력하여 어떤 요청이 실패했는지 확인할 수 있도록 함
    
    else:
        # HTTP 응답 상태 코드가 200(정상)일 때 실행
        soup = BeautifulSoup(response.text, 'html.parser')
        # BeautifulSoup 객체를 생성
        # response.text를 이용해서 HTML 코드를 가져오고 
        # 'html.parser'를 이용해서 HTML 코드를 파싱 -> soup 변수에 파싱된 HTML 코드가 저장됨

        이름 = soup.select_one('.con_title') # 시설명 추출
        주소 = soup.select_one('td[data-th="주소"]') # 주소 추출
        이름 = 이름.text # 태그에서 텍스트만 추출
        주소 = 주소.text # 태그에서 텍스트만 추출
        data.append([f'{i:03d}', 이름, 주소]) # 결과 리스트에 추가
        print(f"olSkey=DNOL0000000000{i:03d}: 이름: {이름}, 주소: {주소}") # 결과 출력

df = pd.DataFrame(data, columns=['주소번호', '이름', '주소']) # 수집한 데이터를 DataFrame으로 변환
df.to_csv('부산광역시 들락날락 주소 모음.csv') # DataFrame을 CSV 파일로 저장
busan_df = pd.read_csv('부산광역시 들락날락 주소 모음.csv') # 저장한 CSV 파일을 다시 읽어옴

len(busan_df.index)
# DataFrame의 행 개수(수집한 들락날락 데이터 개수)를 출력

## 출력 결과 일부
olSkey=DNOL0000000000000: 요청 실패 - 500

olSkey=DNOL0000000000001: 요청 실패 - 500

...

olSkey=DNOL0000000000027: 이름: 다대도서관 들락날락, 주소: 부산시 사하구 다대낙조2길 9

olSkey=DNOL0000000000028: 이름: 더나눔어린이작은도서관 들락날락, 주소: 동구 영초윗길 48(초량동), 장기려기념관 1층

...

olSkey=DNOL0000000000299: 요청 실패 - 500

olSkey=DNOL0000000000300: 요청 실패 - 500

107
- 총 106개소의 들락날락으로 106이 출력되어야 하나 107이 출력됨을 확인

In [ ]:
pd.set_option('display.max_rows', None)  # DataFrame 출력 시 모든 행을 화면에 표시하도록 설정
busan_df # 수집된 데이터를 출력하여 중복되는 데이터 검토

솔바람문화센터 들락날락 중복 확인
- 솔바람문화센터 들락날락,  부산 사하구 하단동 786-1번지 일원
    - 들락날락 정보에서 사하구만 선택, 검색 결과 솔바람문화센터 들락날락(조성중)만 확인 가능. 솔바람문화센터 들락날락은 URL로만 확인 가능.
- 솔바람문화센터 들락날락(조성중),  부산광역시 사하구 에덴공원 내 솔바람문화센터
    - URL 맨 뒷자리 숫자가 218, 219로 1밖에 차이가 나지 않는 것으로 보아 솔바람문화센터 들락날락 정보를 수정하고 수정되지 않은 URL은 삭제하지 않은 것으로 추정.

아래 코드로 중복되는 들락날락 이름 행과 불필요한 열 삭제

In [ ]:
busan_df1 = pd.read_csv('부산광역시 들락날락 주소 모음.csv')
busan_df1.drop(busan_df1[busan_df1['주소'] == '부산 사하구 하단동 786-1번지 일원'].index, inplace=True)
# 솔바람문화센터 들락날락,  부산 사하구 하단동 786-1번지 일원에 해당하는 행 삭제
busan_df1.drop('Unnamed: 0', axis=1, inplace=True) # 'unnamed: 0' 열 삭제
busan_df1.to_csv('부산광역시 들락날락 주소 모음.csv', index=False) # 인덱스 없이 저장

# 수집한 들락날락 주소를 좌표로 변환하기
## 주소를 좌표로 변환하는 오픈API 이용
### 브이월드의 Geocoder API 2.0 레퍼런스 활용
- https://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do


In [ ]:
import pandas as pd
import requests
import time
import random

busan_df2 = pd.read_csv('부산광역시 들락날락 주소 모음.csv')
busan_df2['위도'] = pd.NA
busan_df2['경도'] = pd.NA
# 위도, 경도 컬럼을 추가하고 모두 NaN 값으로 채움

for idx, addr in enumerate(busan_df2['주소']):
    apiurl = 'https://api.vworld.kr/req/address?'
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": addr,
        "format": "json",
        "type": "road", # 도로명 주소로 좌표 변환 시도
        "key": '3BDEB80E-7485-3E95-9639-1E81D0E02869', # 발급받은 API 키를 작은따옴표 사이에 입력
    }
    response = requests.get(apiurl, params=params)
    time.sleep(random.uniform(1, 2))  # 1~2초 대기(과도한 요청으로 인한 서버 부담 방지)
    if response.status_code == 200:
        response_json = response.json()
        if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
            point = response_json['response']['result']['point']
            # 정상 응답이면서 좌표(point) 정보가 있으면 아래 코드 실행

            busan_df2.at[idx, '위도'] = point['y']
            busan_df2.at[idx, '경도'] = point['x']
            # busan_df2 변수의 위도/경도 컬럼에 위도/경도 저장
            
            print(f"인덱스 {idx}, 주소 '{addr}' 출력 성공: 위도 {point['y']}, 경도 {point['x']}")
            # 위도/경도 출력 및 저장이 성공했음을 알기 쉽도록 메시지 출력
        
        else:
            # 도로명 주소로 실패 시 지번 주소로 재시도
            print(f"인덱스 {idx}, 주소 '{addr}' 출력 실패. 재시도")
            params['type'] = 'parcel'
            response = requests.get(apiurl, params=params)
            time.sleep(random.uniform(1, 2))
            if response.status_code == 200:
                response_json = response.json()
                if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
                    point = response_json['response']['result']['point']
                    busan_df2.at[idx, '위도'] = point['y']
                    busan_df2.at[idx, '경도'] = point['x']
                    print(f"인덱스 {idx}, 주소 '{addr}' 재시도 성공: 위도 {point['y']}, 경도 {point['x']}")
                else:
                    # 도로명/지번 주소 모두 실패하면 아래 메시지 출력
                    print(f"인덱스 {idx}, 주소 '{addr}' 출력 실패")

            else: # API 요청 자체가 실패한 경우 아래 메시지 출력
                print(f"인덱스 {idx}, 주소 '{addr}' 재시도 요청 실패: 상태 코드 {response.status_code}")
    else:
        print(f"인덱스 {idx}, 주소 '{addr}' 요청 실패: 상태 코드 {response.status_code}")

위도_개수 = busan_df2['위도'].count()
경도_개수 = busan_df2['경도'].count()
print(f"위도 값 개수: {위도_개수}")
print(f"경도 값 개수: {경도_개수}")
# 위도와 경도 컬럼에서 NaN이 아닌 값(즉, 실제 좌표가 있는 데이터)의 개수 출력

## 출력 결과 일부
인덱스 0, 주소 '부산시 사하구 다대낙조2길 9' 출력 성공: 위도 35.050427010, 경도 128.964700408

인덱스 1, 주소 '동구 영초윗길 48(초량동), 장기려기념관 1층' 출력 성공: 위도 35.118525186, 경도 129.032654093

...

인덱스 55, 주소 '연제구 연산동 1829번지 외 7필지' 출력 실패. 재시도
인덱스 55, 주소 '연제구 연산동 1829번지 외 7필지' 재시도 성공: 위도 35.17295582544455, 경도 129.09289429533436

...

인덱스 104, 주소 '동래구 우장춘로 117' 출력 성공: 위도 35.217858768, 경도 129.074224891
인덱스 105, 주소 '부산진구 가야대로 734' 출력 성공: 위도 35.156745057, 경도 129.052320235
위도 값 개수: 103
경도 값 개수: 103
- 106개 주소 중 3개 주소의 좌표 출력 실패 확인 후 아래 코드를 실행하여 누락된 좌표의 주소 확인

In [ ]:
busan_df2 # 수집된 데이터를 출력하여 누락되는 데이터 검토

### 106개 주소 중 3개 출력 실패 확인 후 별도 출력
1. 기장어린이도서관 들락날락: 기장군 기장읍 처성동로 126번길 13-5
- 검색 결과 처성동로가 아닌 차성동로로 확인
2. 솔바람문화센터 들락날락(조성중): 부산광역시 사하구 에덴공원 내 솔바람문화센터
- 조성중인 들락날락 주소는 검색해도 나오지 않았습니다. 솔바람문화센터 들락날락(조성중) 주소는 들락날락 사이트에서 지도에 표시된 위치정보를 참고하여 에덴공원 내 에덴유원지 지번 주소로 변경하고 좌표로 변환했습니다.
3. 국립해양박물관내 어린이해양도서관 들락날락(조성중): 부산광역시 영도구 해양로310번길 45 국립해양박물관내 어린이해양도서관
    - 국립해양박물관은 해양로310번길이 아닌 해양로301번길로 확인.
    - 조성중인 들락날락은 검색해도 나오지 않았습니다. 그래서 어린이해양도서관 위치는 국립해양박물관 주소를 기준으로 좌표로 변환했습니다.

### 수정한 주소는 address2에, 수정 전 주소는 address1에 할당하여 수정한 주소로 좌표 변환 및 수정 전 주소를 수정 후 주소로 변경

In [ ]:
import pandas as pd
import requests

pd.set_option('display.max_rows', None)
address1 = ['기장군 기장읍 처성동로 126번길 13-5', '부산광역시 사하구 에덴공원 내 솔바람문화센터', '부산광역시 영도구 해양로310번길 45 국립해양박물관내 어린이해양도서관']
# 수정 전 주소 목록 (좌표 변환이 실패했던 원본 주소)
address2 = ['기장군 기장읍 차성동로 126번길 13-5', '부산 사하구 하단동 786-100', '부산광역시 영도구 해양로301번길 45']
# 수정 후 주소 목록

for old_addr, new_addr in zip(address1, address2):
    # address1과 address2를 동시에 순회하며
    # old_addr(수정 전 주소), new_addr(수정 후 주소)로 사용
    apiurl = 'https://api.vworld.kr/req/address?'
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": new_addr, # 수정된 주소로 좌표 변환 요청
        "format": "json",
        "type": "road",
        "key": '3BDEB80E-7485-3E95-9639-1E81D0E02869',
    }
    response = requests.get(apiurl, params=params)
    time.sleep(random.uniform(1, 2))
    if response.status_code == 200:
        response_json = response.json()
        if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
            point = response_json['response']['result']['point']
            # 정상 응답이면 좌표(point) 정보 추출

            busan_df2.loc[busan_df2['주소'] == old_addr, '위도'] = point['y']
            busan_df2.loc[busan_df2['주소'] == old_addr, '경도'] = point['x']
            # busan_df2에서 old_addr(수정 전 주소)에 해당하는 행의 위도/경도 값을 새 좌표로 수정
            
            busan_df2.loc[busan_df2['주소'] == old_addr, '주소'] = new_addr
            # 그리고 주소 컬럼도 old_addr에서 new_addr로 수정
            print(f"주소 '{old_addr}' → '{new_addr}' 좌표 수정 완료: 위도 {point['y']}, 경도 {point['x']}")
        else:
            # 도로명 주소로 실패 시 지번 주소로 재시도
            print(f"주소 '{new_addr}' 좌표 변환 실패. 재시도")
            params['type'] = 'parcel'
            response = requests.get(apiurl, params=params)
            time.sleep(random.uniform(1, 2))
            if response.status_code == 200:
                response_json = response.json()
                if 'response' in response_json and 'result' in response_json['response'] and 'point' in response_json['response']['result']:
                    point = response_json['response']['result']['point']
                    # 정상 응답이면 좌표(point) 정보 추출

                    busan_df2.loc[busan_df2['주소'] == old_addr, '위도'] = point['y']
                    busan_df2.loc[busan_df2['주소'] == old_addr, '경도'] = point['x']
                    # busan_df2에서 old_addr(수정 전 주소)에 해당하는 행의 위도/경도 값을 새 좌표로 수정
                    
                    busan_df2.loc[busan_df2['주소'] == old_addr, '주소'] = new_addr
                    # 그리고 주소 컬럼도 old_addr에서 new_addr로 수정
                    print(f"주소 '{old_addr}' → '{new_addr}' 좌표 수정 완료: 위도 {point['y']}, 경도 {point['x']}")
                else:
                    # 도로명/지번 주소 모두 실패하면 아래 메시지 출력
                    print(f"인덱스 {idx}, 주소 '{addr}' 출력 실패")

            else: # API 요청 자체가 실패한 경우 아래 메시지 출력
                print(f"인덱스 {idx}, 주소 '{addr}' 재시도 요청 실패: 상태 코드 {response.status_code}")
    else:
        print(f"인덱스 {idx}, 주소 '{addr}' 요청 실패: 상태 코드 {response.status_code}")
# 결과적으로 busan_df2에는 수정 전 주소가 수정 후 주소로 바뀌고, 위도/경도도 정확하게 갱신됨

busan_df2
# 최종적으로 수정된 데이터프레임을 출력하여 검토

## 출력 결과

주소 '기장군 기장읍 처성동로 126번길 13-5' → '기장군 기장읍 차성동로 126번길 13-5' 좌표 수정 완료: 위도 35.249684877, 경도 129.217304190
주소 '부산 사하구 하단동 786-100' 좌표 변환 실패. 재시도
주소 '부산광역시 사하구 에덴공원 내 솔바람문화센터' → '부산 사하구 하단동 786-100' 좌표 수정 완료: 위도 35.109272242716486, 경도 128.96289622481936
주소 '부산광역시 영도구 해양로310번길 45 국립해양박물관내 어린이해양도서관' → '부산광역시 영도구 해양로301번길 45' 좌표 수정 완료: 위도 35.078709425, 경도 129.080199790

- 이후 busan_df2 출력

In [ ]:
busan_df2.to_csv('부산광역시_좌표추가.csv', index=False, encoding='utf-8')
# 정상적으로 수정됐음을 확인 후 다른 CSV 파일에 저장